# Run FlowSOM for pixel and cell clustering

In [ ]:
import harpy as hp
from harpy.datasets import pixie_example
from harpy.utils._keys import ClusteringKey

## 1. Load example dataset

In [ ]:
sdata = pixie_example(["fov0", "fov1"])
sdata

In [3]:
channels = [
    "CD3",
    "CD4",
    "CD8",
    "CD14",
    "CD20",
    "CD31",
    "CD45",
    "CD68",
    "CD163",
    "CK17",
    "Collagen1",
    "Fibronectin",
    "ECAD",
    "HLADR",
    "SMA",
    "Vim",
]

In [ ]:
hp.pl.plot_image(sdata, img_layer="raw_image_fov0", to_coordinate_system="fov0", channel="CD3", figsize=(4, 4))

In [ ]:
hp.pl.histogram(
    sdata,
    img_layer="raw_image_fov0",
    channel="CD3",
    bins=100,
    fig_kwargs={
        "figsize": (4, 4),
    },
)

## 2. Preprocess

In [ ]:
sdata = hp.im.pixel_clustering_preprocess(
    sdata,
    img_layer=["raw_image_fov0", "raw_image_fov1"],
    output_layer=["raw_image_fov0_processed", "raw_image_fov1_processed"],
    channels=channels,
    chunks=2048,
    persist_intermediate=True,  # set to False if you have multiple images, and if they are large.
    overwrite=True,
    sigma=2.0,
)

In [ ]:
hp.pl.plot_image(
    sdata, img_layer="raw_image_fov0_processed", to_coordinate_system="fov0", channel="CD3", figsize=(4, 4)
)

In [ ]:
hp.pl.histogram(
    sdata,
    img_layer="raw_image_fov0_processed",
    channel="CD3",
    bins=100,
    fig_kwargs={
        "figsize": (4, 4),
    },
)

## 3. Pixel clustering

In [ ]:
import flowsom as fs
from dask.distributed import Client, LocalCluster

work_with_client = False

if work_with_client:
    # client example
    cluster = LocalCluster(
        n_workers=1,
        threads_per_worker=10,
    )

    client = Client(cluster)
else:
    client = None

batch_model = fs.models.BatchFlowSOMEstimator

sdata, fsom, mapping = hp.im.flowsom(
    sdata,
    img_layer=["raw_image_fov0_processed", "raw_image_fov1_processed"],
    output_layer_clusters=[
        "raw_image_fov0_flowsom_clusters",
        "raw_image_fov1_flowsom_clusters",
    ],  # we need output_cluster_layer and output_meta_cluster_layer --> these will both be labels layers
    output_layer_metaclusters=[
        "raw_image_fov0_flowsom_metaclusters",
        "raw_image_fov1_flowsom_metaclusters",
    ],
    n_clusters=20,
    random_state=111,
    chunks=512,
    client=client,
    model=batch_model,
    num_batches=10,
    xdim=10,
    ydim=10,
    z_score=True,
    z_cap=3,
    persist_intermediate=True,
    overwrite=True,
)

In [ ]:
sdata = hp.tb.cluster_intensity(
    sdata,
    mapping=mapping,
    img_layer=["raw_image_fov0_processed", "raw_image_fov1_processed"],
    labels_layer=["raw_image_fov0_flowsom_clusters", "raw_image_fov1_flowsom_clusters"],
    to_coordinate_system=["fov0", "fov1"],
    output_layer="counts_clusters",
    overwrite=True,
)

## 4. Visualization of pixel clusters and metaclusters

In [ ]:
hp.pl.pixel_clusters(
    sdata,
    labels_layer="raw_image_fov0_flowsom_clusters",
    figsize=(10, 10),
    to_coordinate_system="fov0",
    render_labels_kwargs={"alpha": 1},
)

hp.pl.pixel_clusters(
    sdata,
    labels_layer="raw_image_fov0_flowsom_metaclusters",
    figsize=(10, 10),
    to_coordinate_system="fov0",
    render_labels_kwargs={"alpha": 1},
)

## 5. Heatmap of channel intensity per cluster and metacluster

In [ ]:
for _metaclusters in [True, False]:
    hp.pl.pixel_clusters_heatmap(
        sdata,
        table_layer="counts_clusters",
        figsize=(25, 8),
        fig_kwargs={"dpi": 100},
        linewidths=0.001,
        metaclusters=_metaclusters,
        z_score=True,
    )

## 6. Spatial pixel neighbors

In [ ]:
import numpy as np
import squidpy as sq

key_added = "cluster_id"

adata = hp.tb.spatial_pixel_neighbors(
    sdata,
    labels_layer="raw_image_fov0_flowsom_metaclusters",
    key_added=key_added,
    mode="most_frequent",
    grid_type="hexagon",
    size=20,
    subset=None,
)

adata.uns[f"{key_added}_nhood_enrichment"]["zscore"] = np.nan_to_num(
    adata.uns[f"{key_added}_nhood_enrichment"]["zscore"]
)
sq.pl.nhood_enrichment(adata, cluster_key=key_added, method="ward", mode="zscore", figsize=(8, 8))

## 7. Cell clustering

In [ ]:
batch_model = fs.models.BatchFlowSOMEstimator

sdata, fsom = hp.tb.flowsom(
    sdata,
    labels_layer_cells=["label_whole_fov0", "label_whole_fov1"],
    labels_layer_clusters=[
        "raw_image_fov0_flowsom_metaclusters",
        "raw_image_fov1_flowsom_metaclusters",
    ],  # here you could also choose "ark_pixel_som_cluster"
    output_layer="table_cell_clustering_flowsom",
    chunks=512,
    model=batch_model,
    num_batches=10,
    random_state=100,
    overwrite=True,
)

In [ ]:
sdata.pl.render_labels("label_whole_fov0", table_name="table_cell_clustering_flowsom", color="metaclustering").pl.show(
    figsize=(5, 5)
)

In [ ]:
# "table_cell_clustering_flowsom" is annotated by segmentation masks, so they can also be visualised using napari-spatialdata
sdata["table_cell_clustering_flowsom"].uns["spatialdata_attrs"]

# from napari_spatialdata import Interactive

# Interactive(sdata)

## Optional export to a .csv format that can be used for visualization using the ark analysis [gui](https://github.com/angelolab/ark-analysis/blob/main/templates/2_Pixie_Cluster_Pixels.ipynb)

In [ ]:
# weighted channel average for visualization -> calculate this on the flowsom clustered matrix
sdata = hp.tb.weighted_channel_expression(
    sdata,
    table_layer_cell_clustering="table_cell_clustering_flowsom",
    table_layer_pixel_cluster_intensity="counts_clusters",
    output_layer="table_cell_clustering_flowsom",
    clustering_key=ClusteringKey._METACLUSTERING_KEY,
    overwrite=True,
)

In [ ]:
from harpy.table.cell_clustering._utils import _export_to_ark_format as _export_to_ark_format_cells
from harpy.table.pixel_clustering._cluster_intensity import _export_to_ark_format as _export_to_ark_format_pixels

df = _export_to_ark_format_pixels(adata=sdata["counts_clusters"], output=None)
(
    df_cell_som_cluster_count_avg,
    df_cell_som_cluster_channel_avg,
    df_cell_meta_cluster_channel_avg,
) = _export_to_ark_format_cells(sdata, table_layer="table_cell_clustering_flowsom", output=None)
df.head()

In [ ]:
df_cell_meta_cluster_channel_avg.head()